In [ ]:
import solcx
solcx.get_installed_solc_versions()
solcx.get_installable_solc_versions()
# solcx.install_solc(version="0.7.6", show_progress=False, solcx_binary_path=None)

In [ ]:
import pandas as pd
df = pd.read_csv(r"smart-contract-sanctuary-ethereum_contracts_mainnet_contracts_deduplicate_sample.csv", encoding="utf-8", header=0)

In [ ]:
import os
import logging
import traceback

def getFilePathWithSameAddress(root_path,address):
    file_dir_path = os.path.join(root_path, str(address[2:4]).lower())
    path_arr = ""
    try:
        file_names = os.listdir(file_dir_path)
        path_arr = [os.path.join(file_dir_path,f) for f in file_names if (m := re.match(str(address[2:]).lower()+".*",str(f).lower()))]
    except:
        logging.warning(traceback.format_exc())
    if len(path_arr)==0:
        logging.debug("INFO: Cannot find contract {0} files.".format(address))
    return path_arr

In [ ]:
logging.basicConfig(format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s',
                        level=logging.DEBUG,
                        filename= os.path.join(os.path.abspath("."),'compile_check.ipynb.log'),
                        filemode='w') 
path_input_dataset = r"./smart-contract-sanctuary-ethereum/contracts/mainnet"
path_output_csv = r"smart-contract-sanctuary-ethereum_contracts_mainnet_contracts_compile_check.csv"

list_error = []
list_more_than_one_file = []

for index, row in df.iterrows():
    path_arr = getFilePathWithSameAddress(path_input_dataset,row["address"])
    if len(path_arr) > 1:
        list_more_than_one_file.append(row["address"])
        break
    try:
        print(str(row["compiler"]).strip().replace("v",""))
        print(path_arr[0])
        solcx.compile_files(path_arr[0],output_values=["abi"],solc_version=str(row["compiler"]).strip().replace("v",""))
    except:
        list_error.append(index)
        logging.warning(traceback.format_exc())
        print("ERROR:Index {0}, {1} cannot be compiled successfully".format(index,path_arr))
print("The total number of contracts that has more than one file: {0}".format(len(list_more_than_one_file)))
print("The total number of contracts that failed to compile: {0}".format(len(list_error)))